In [1]:
from scripting.loader import db_setup
from scripting.loader.models import site, company, offer_data, geography, offer_geography, offer, salary, offer_salary
from scripting.parser.pracuj_pl.page_parser import PracujPLParser

In [2]:
parser = PracujPLParser('pracuj.pl')

with open('/Users/ulakruts/Personal Project/Praca_Inzynierska/MJ/test.html', 'r') as f:
    content = f.read()

In [3]:
page_data = parser.parse(content, 'huj')

In [4]:
page_data

{'url': 'huj',
 'site': 'pracuj.pl',
 'company_name': 'alior bank',
 'company_url': None,
 'company_description': None,
 'offer_title': 'specjalista ds. portalu (ux i ui)',
 'position_level': [{'id': 4,
   'name': 'specjalista (Mid / Regular)',
   'pracujPlName': 'specjalista (Mid / Regular)'}],
 'technology_list': {'required': ['HTML'], 'optional': None},
 'offer_description': 'alior bank konsekwentnie pracuje na miano innowatora. stały rozwój bankowości mobilnej i regularne wdrażanie najbardziej nowatorskich na rynku rozwiązań wymaga potężnego zaplecza it - zarówno w zakresie przygotowania aplikacji i systemów, jak i oceny ryzyka oraz kontroli bezpieczeństwa danych. aktualnie w pionie it alior banku zatrudniamy ponad 700 specjalistów. wykorzystują oni 15 różnych platform, 14 frameworków, ponad 50 różnych technologii w obszarze infrastruktury oraz 10 baz danych. wdrażamy aktualnie nowe systemy, takie jak kafka, eureka, kubernetes. pracujemy zarówno w klasycznym modelu projektowym, jak

In [15]:
site.Site.create(name='pracuj.paal', url='https://pracuj.pl')

<Site(id=2, name='pracuj.paal', domain='None')>

In [19]:
company.Company.create(name='pracuj.paal', url='https://pracuj.pl', business_type='IT', description='IT')

<Company(id=1, name='pracuj.paal', url='https://pracuj.pl', business_type='IT')>

In [12]:
site_session = site.Session()
site_session.add(p_site)
site_session.commit()

In [13]:
site_session.close()

In [6]:
def test_offer_data():
    session =  db_setup.Session()
    test_offer_data = {
        "data": {
            "offer_description": "alior bank konsekwentnie pracuje na miano innowatora...",
            "position_level": [{"id": 4, "name": "specjalista (Mid / Regular)", "pracujPlName": "specjalista (Mid / Regular)"}],
            "work_type": ["hybrid"]
        },
        "requirements": "masz wysokie umiejętności analityczne, jest ambitna, otwarta на nowe wyzwania...",
        "responsibilities": "inicjowanie projektów в ramach rozwoju portalu – w celu zwiększenia sprzedaży...",
        "original_language": "pl",
        "translated_data": {"offer_description_translated": "translated text"}
    }

    try:
        # Проверка метода create
        print("Creating a new OfferData record...")
        offer_data_record = offer_data.OfferData.create(**test_offer_data)
        if offer_data_record:
            print(f"OfferData created with ID: {offer_data_record.id}")

        # Проверка метода exists
        print("Checking if OfferData record exists...")
        exists = offer_data.OfferData.exists(data=test_offer_data["data"])
        print(f"OfferData exists: {exists}")

        # Проверка метода update
        if offer_data_record:
            print("Updating OfferData record...")
            updated_record = offer_data.OfferData.update(
                record_id=offer_data_record.id,
                requirements="Updated requirements",
                responsibilities="Updated responsibilities"
            )
            print(f"Updated OfferData: {updated_record.requirements}, {updated_record.responsibilities}")

        # Проверка выборки из базы
        print("Querying OfferData records...")
        all_records = session.query(offer_data.OfferData).all()
        for record in all_records:
            print(f"OfferData ID: {record.id}, Requirements: {record.requirements}")

    except Exception as e:
        print(f"An error occurred: {e}")
        session.rollback()
    finally:
        session.close()


# Запуск теста
test_offer_data()


Creating a new OfferData record...
OfferData created with ID: 1
Checking if OfferData record exists...
An error occurred: (psycopg2.errors.UndefinedFunction) operator does not exist: jsonb = json
LINE 3: WHERE offer_data.data = '{"offer_description": "alior bank k...
                              ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: SELECT offer_data.data AS offer_data_data, offer_data.requirements AS offer_data_requirements, offer_data.responsibilities AS offer_data_responsibilities, offer_data.original_language AS offer_data_original_language, offer_data.translated_data AS offer_data_translated_data, offer_data.id AS offer_data_id, offer_data.create_date AS offer_data_create_date, offer_data.update_date AS offer_data_update_date 
FROM offer_data 
WHERE offer_data.data = %(data_1)s::JSON 
 LIMIT %(param_1)s]
[parameters: {'data_1': '{"offer_description": "alior bank konsekwentnie pracuje na miano innowatora..

In [12]:
# from models.geography import Geography
# from models.offer import Offer
# from models.offer_geography import OfferGeography
# from database.db_setup import SessionLocal


def test_offer_geography_creation():
    session = Session()

    try:
        print("Creating a test offer...")
        offer_data =  offer.Offer.create(
            url="test_offer_url",
            name="Test Offer",
            position_level="Mid",
            data_id=-1,
            site_name="Test Site",
            company_name="Test Company",
            active=True
        )
        if not offer_data:
            print("Failed to create Offer. Check the create method for Offer.")
            return

        print(f"Offer created with ID: {offer_data.id}, URL: {offer_data.url}")

        print("Creating a test geography...")
        geography_data =  geography.Geography.create(
            city="Test City",
            country="Test Country"
        )
        if not geography_data:
            print("Failed to create Geography. Check the create method for Geography.")
            return

        print(f"Geography created with ID: {geography_data.id}, City: {geography_data.city}, Country: {geography_data.country}")

        print("Creating a test OfferGeography...")
        offer_geography_data = offer_geography.OfferGeography.create(
            offer_url="test_offer_url",
            city="Test City",
            country="Test Country"
        )
        if offer_geography_data:
            print(f"OfferGeography created: OfferID={offer_geography_data.id_offer}, GeographyID={offer_geography_data.id_geography}")
        else:
            print("Failed to create OfferGeography. Check the create method for OfferGeography.")

        print("Testing duplicate OfferGeography creation...")
        duplicate_offer_geography_data = offer_geography.OfferGeography.create(
            offer_url="test_offer_url",
            city="Test City",
            country="Test Country"
        )
        if not duplicate_offer_geography_data:
            print("Duplicate OfferGeography creation prevented successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
        session.rollback()
    finally:
        session.close()


test_offer_geography_creation()


Creating a test offer...
Offer created with ID: 2, URL: test_offer_url
Creating a test geography...
Geography created with ID: 2, City: Test City, Country: Test Country
Creating a test OfferGeography...
OfferGeography created: OfferID=2, GeographyID=2
Testing duplicate OfferGeography creation...
OfferGeography with offer URL 'test_offer_url' and geography 'Test City, Test Country' already exists!
Duplicate OfferGeography creation prevented successfully!


In [11]:


def test_offer_creation():
    session = Session()

    try:
        # Создание записи в Site
        print("Creating a test site...")
        site_data = site.Site.create(
            name="Test Site",
            domain="testsite.com"
        )
        if not site_data:
            print("Failed to create Site. Check the create method for Site.")
            return

        print(f"Site created with ID: {site_data.id}, Name: {site_data.name}, Domain: {site_data.domain}")

        # Создание записи в Company
        print("Creating a test company...")
        company_data = company.Company.create(
            name="Test Company",
            url="https://testcompany.com",
            business_type="IT",
            description="A test company for testing."
        )
        if not company_data:
            print("Failed to create Company. Check the create method for Company.")
            return

        print(f"Company created with ID: {company_data.id}, Name: {company_data.name}, URL: {company_data.url}")

        # Создание записи в Offer
        print("Creating a test offer...")
        offer_data = offer.Offer.create(
            url="https://testoffer.com",
            name="Test Offer",
            position_level="Mid",
            data_id=-1,
            site_name="Test Site",
            company_name="Test Company",
            active=True
        )
        if offer_data:
            print(f"Offer created: ID={offer_data.id}, URL={offer_data.url}, Name={offer_data.name}, Position Level={offer_data.position_level}")
        else:
            print("Failed to create Offer. Check the create method for Offer.")

        # Повторное создание для проверки уникальности
        print("Testing duplicate Offer creation...")
        duplicate_offer_data = offer.Offer.create(
            url="https://testoffer.com",
            name="Duplicate Offer",
            position_level="Senior",
            data_id=-1,
            site_name="Test Site",
            company_name="Test Company",
            active=True
        )
        if not duplicate_offer_data:
            print("Duplicate Offer creation prevented successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
        session.rollback()
    finally:
        session.close()


test_offer_creation()


Creating a test site...
Site created with ID: 1, Name: Test Site, Domain: testsite.com
Creating a test company...
Company created with ID: 1, Name: Test Company, URL: https://testcompany.com
Creating a test offer...
Offer created: ID=1, URL=https://testoffer.com, Name=Test Offer, Position Level=Mid
Testing duplicate Offer creation...
Offer with URL 'https://testoffer.com' already exists!
Duplicate Offer creation prevented successfully!


In [3]:
from scripting.loader.db_setup import Session


def test_offer_salary_creation():
    session = Session()

    try:
        print("Creating a test offer...")
        offer_data = offer.Offer.create(
            url="test_ofasassssfqefer_url",
            name="Test Offer",
            position_level="Mid",
            data_id=-1,
            site_name="Test Site",
            company_name="Test Company",
            active=True
        )
        if not offer_data:
            print("Failed to create Offer. Check the create method for Offer.")
            return

        print(f"Offer created with ID: {offer_data.id}, URL: {offer_data.url}")

        print("Creating a test salary...")
        salary_data = salary.Salary.create(
            value="5000-7000 PLN",
            contract_type="Full-time"
        )
        if not salary_data:
            print("Failed to create Salary. Check the create method for Salary.")
            return

        print(f"Salary created with ID: {salary_data.id}, Value: {salary_data.value}, Contract Type: {salary_data.contract_type}")

        print("Creating a test OfferSalary...")
        offer_salary_data = offer_salary.OfferSalary.create(
            offer_url="test_offer_url",
            salary_value="5000-7000 PLN",
            salary_contract_type="Full-time"
        )
        if offer_salary_data:
            print(f"OfferSalary created: ID={offer_salary_data.id}, OfferID={offer_salary_data.id_offer}, SalaryID={offer_salary_data.id_salary}")
        else:
            print("Failed to create OfferSalary. Check the create method for OfferSalary.")

        print("Testing duplicate OfferSalary creation...")
        duplicate_offer_salary_data = offer_salary.OfferSalary.create(
            offer_url="test_offer_url",
            salary_value="5000-7000 PLN",
            salary_contract_type="Full-time"
        )
        if not duplicate_offer_salary_data:
            print("Duplicate OfferSalary creation prevented successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
        session.rollback()
    finally:
        session.close()


test_offer_salary_creation()

Creating a test offer...
Offer with URL 'test_ofasassssfqefer_url' already exists!
Offer created with ID: 4, URL: test_ofasassssfqefer_url
Creating a test salary...
Salary already exists!
Salary created with ID: 1, Value: 5000-7000 PLN, Contract Type: Full-time
Creating a test OfferSalary...
OfferSalary with salary '5000-7000 PLN' and offer 'test_offer_url' already exists!
Failed to create OfferSalary. Check the create method for OfferSalary.
Testing duplicate OfferSalary creation...
OfferSalary with salary '5000-7000 PLN' and offer 'test_offer_url' already exists!
Duplicate OfferSalary creation prevented successfully!
